<a href="https://colab.research.google.com/github/ritika-0111/Bias-Toxic-Classification/blob/main/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounted Google Drive to access Data prepared from the Jigsaw *Dataset*

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Setting Configuration to fetch Kaggle API

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

In [3]:
#changing the working directory
%cd /content/gdrive/MyDrive/Kaggle

/content/gdrive/MyDrive/Kaggle


Dataset downloaded through Kaggle API

In [4]:
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

identity_individual_annotations.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
all_data.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_public_expanded.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_private_expanded.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
toxicity_individual_annotations.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


Unzipping the zip files and Deleting the zip files


In [9]:
!unzip \*.zip  && rm *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


#### Import Libraries

In [10]:
import numpy as np
import pandas as pd


#### Read Dataset

In [7]:
df1 = pd.read_csv('/content/gdrive/MyDrive/Kaggle/train.csv')

In [8]:
df1.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [13]:
df1.shape

(1804874, 45)

#### Handle Missing Values
Drop missing values

In [14]:
df1.dropna(inplace=True)

On dropping the missing values, our dataset has reduced to (235087, 45)
from (1804874, 45)


In [15]:
df1.shape

(235087, 45)

Identiting Required columns for creating a new column Gender

In [16]:
identity_columns = ['male', 'female']

Replacing all the values with value greater than or equal to 0.5 with True and all the values less than 0.5 as False

In [17]:
for column in identity_columns + ['target']:
    df1[column] = np.where(df1[column] >= 0.5, True, False)

Created new column Gender by merging column male and column female

In [18]:
df1['gender'] = np.where((df1['male'] == True), "m" , "f")

For our implementation, we use only the *comment_text* column, which we rename to *Comment* and contains the textual information, and the *gender* column whic is renamed as *Gender*, which represents the sex i.e. male or female.

In [19]:
df2 = df1[['gender', 'comment_text']]

In [20]:
df2.columns = ['Gender', "Comment"]

In [21]:
df2.head()

,Gender,Comment
11,m,This is a great story. Man. I wonder if the pe...
31,f,Yet call out all Muslims for the acts of a few...
51,f,Because the people who drive cars more are the...
58,f,Mormons have had a complicated relationship wi...
111,f,I'm doing the same thing! :)


In [23]:
df2.shape

(235087, 2)

In [24]:
df2['Gender'].value_counts()

f    210462
m     24625
Name: Gender, dtype: int64

In [25]:
df2 = df2[df2['Gender'] != 'Other']


We need to represent data as numeric values for the model. Here we create a new column Gender_Label that encodes the information from the Gender column into numeric values.

In [27]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df2['Gender_Label'] = enc.fit_transform(df2['Gender'])

In [28]:
df2.iloc[4]['Comment']

"I'm doing the same thing! :)"

Text Preprocessing
By removing punctuation, removing the masked information , removing extra spaces and finally normalize everything to lowercase.

In [29]:
import string

table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
df2['Comment'] = df2['Comment'].str.translate(table)
df2['Comment'] = df2['Comment'].str.replace('X+', '')
df2['Comment'] = df2['Comment'].str.replace(' +', ' ')
df2['Comment'] = df2['Comment'].str.lower()
df2['Comment'] = df2['Comment'].str.strip()

In [30]:
df2.iloc[4]['Comment']

'i m doing the same thing'

In [31]:
lengths = [len(df2.iloc[i]['Comment'].split()) for i in range(len(df2))]
print(max(lengths))
print(min(lengths))

220
0


In [32]:
df2 = df2[[l >= 1 for l in lengths]]

In [33]:
df2.shape

(235076, 3)

In [34]:
pd.DataFrame(df2['Gender'].value_counts())

,Gender
f,210451
m,24625


#### Save Data
We then save the preprocessed dataset, and another one corresponding to a 10% sample.

In [35]:
df2.to_csv('/content/gdrive/MyDrive/Kaggle/data_prepared.csv', index=False)

In [36]:
df2.sample(n=int(len(df2)*0.1), random_state=111).to_csv('/content/gdrive/MyDrive/Kaggle/data_sample_prepared.csv', index=False)

Our Data Sample is prepared, now we will work on this data to perform text classification by fine-tuning a BERT-based model.